In [ ]:
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# Se define la ruta para la base de datos local SQLite
project_name = Path.cwd().parent.parent.name
data_projetc_path = Path.home().joinpath(f'Dropbox/La Casa Del Carpintero/Departamento de Programación/data_projects_git/data_{project_name}')
db_file = 'comisiones.db'
db_file_path_str = str(data_projetc_path.joinpath(db_file))

# Se crea el objeto engine para trabajarlo con los metodos de pandas
engine = create_engine(f'sqlite:///{db_file_path_str}')

In [ ]:
file01 = 'clientes_regalos'
file01_path_str = str(data_projetc_path.joinpath(f'comisiones/{file01}.xlsx'))
clientes_regalos = pd.read_excel(file01_path_str, dtype_backend='numpy_nullable')

file02 = 'refacturaciones'
file02_path_str = str(data_projetc_path.joinpath(f'costo_ventas/{file02}.xlsx'))
refacturaciones = pd.read_excel(file02_path_str, dtype_backend='numpy_nullable')

In [ ]:
with engine.connect() as conn, conn.begin():

    cobranza_enero = pd.read_sql_table('cobranza_1_2024', conn, dtype_backend='numpy_nullable')
    cobranza_febrero = pd.read_sql_table('cobranza_2_2024', conn, dtype_backend='numpy_nullable')
    cobranza_marzo = pd.read_sql_table('cobranza_3_2024', conn, dtype_backend='numpy_nullable')
    cobranza_abril = pd.read_sql_table('cobranza_4_2024', conn, dtype_backend='numpy_nullable')
    cobranza_mayo = pd.read_sql_table('cobranza_5_2024', conn, dtype_backend='numpy_nullable')
    cobranza_junio = pd.read_sql_table('cobranza_6_2024', conn, dtype_backend='numpy_nullable')
    cobranza_julio = pd.read_sql_table('cobranza_7_2024', conn, dtype_backend='numpy_nullable')
    cobranza_año = pd.read_sql_table('cobranza_año_2024', conn, dtype_backend='numpy_nullable')

    costo_ventas_enero = pd.read_sql_table('costo_ventas_1_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_febrero = pd.read_sql_table('costo_ventas_2_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_marzo = pd.read_sql_table('costo_ventas_3_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_abril = pd.read_sql_table('costo_ventas_4_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_mayo = pd.read_sql_table('costo_ventas_5_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_junio = pd.read_sql_table('costo_ventas_6_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_julio = pd.read_sql_table('costo_ventas_7_2024', conn, dtype_backend='numpy_nullable')
    
    datos_com_2023 = pd.read_sql_table('datos_com_2023', conn, dtype_backend='numpy_nullable')
    bonos_limites_comisiones = pd.read_sql_table('bonos_limites_comisiones', conn, dtype_backend='numpy_nullable')

engine.dispose()

costo_ventas_list = [costo_ventas_enero, costo_ventas_febrero, costo_ventas_marzo, costo_ventas_abril, costo_ventas_mayo, costo_ventas_junio, costo_ventas_julio]
costo_ventas_año = pd.concat(costo_ventas_list)

In [ ]:
def comisiones_mes_2024_func(mes:int, costo_ventas_list:list) -> pd.DataFrame:

    df = costo_ventas_list[mes-1]
    mes_df = df['invoice_date'].dt.month.unique()[0]

    df['mes'] = mes_df

    gerentes_list = ['Carlos Manuel Guzmán Rios', 'Mariana Araceli Carvajal Flores', 'Rosario Martinez Zarate']

    comisiones_2024_vend = (df.loc[
        ~df['salesperson_name'].isin(gerentes_list)
        ,
                [
                    'salesperson_id',
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('salesperson_id')
            .agg({
                'salesperson_name':['first'], 
                'business_model':['first'], 
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'salesperson_name',
                    'business_model',
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger = pd.DataFrame(
                [
                    [206, 'Mariana Araceli Carvajal Flores','Piso', mes_df],
                    [208, 'Carlos Manuel Guzmán Rios','CE', mes_df],
                    [209, 'Rosario Martinez Zarate', 'Piso', mes_df],
                ],
                columns = 
                    [
                        'salesperson_id',
                        'salesperson_name',
                        'business_model',
                        'mes'
                    ]
            )

    comisiones_2024_ger_values = (df.loc[:,
                [
                    'sale_team_description',
                    'mes',
                    'price_subtotal',
                    'utilidad_partida_$_after_cc'
                ]
            ].groupby('sale_team_description')
            .agg({
                'mes':['first'], 
                'price_subtotal':['sum'],
                'utilidad_partida_$_after_cc':['sum']
                })
            .set_axis(
                [
                    'mes',
                    'facturacion',
                    'utilidad'
                ], axis='columns')
            .reset_index()
    )


    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Carlos Manuel Guzmán Rios'
            , 
            ['facturacion', 'utilidad']
        ] = (comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'].isin(['A1 CE', 'A2 CE'])
                ]
                .groupby('mes')
                .sum()
                [['facturacion', 'utilidad']]
                .values
            )

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Mariana Araceli Carvajal Flores'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A1 Piso'
                ][['facturacion', 'utilidad']].values

    comisiones_2024_ger.loc[
            comisiones_2024_ger['salesperson_name'] == 'Rosario Martinez Zarate'
            , 
            ['facturacion', 'utilidad']
        ] = comisiones_2024_ger_values[
                    comisiones_2024_ger_values['sale_team_description'] == 'A2 Piso'
                ][['facturacion', 'utilidad']].values


    comisiones_2024 = pd.concat([comisiones_2024_ger, comisiones_2024_vend], ignore_index=True)


    for i in range(len(comisiones_2024)):

        if comisiones_2024.iloc[i]['salesperson_name'] in gerentes_list:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == True)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_2024.iloc[i]['utilidad'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_2024.iloc[i]['utilidad'])
            ]

        else:
            
            bonos_limites_comisiones_line = bonos_limites_comisiones.loc[
                (bonos_limites_comisiones['business_model'] == comisiones_2024.iloc[i]['business_model'])
                & (bonos_limites_comisiones['is_gerente'] == False)
                & (bonos_limites_comisiones['lim_inf'] <= comisiones_2024.iloc[i]['utilidad'])
                & (bonos_limites_comisiones['lim_sup'] >= comisiones_2024.iloc[i]['utilidad'])
            ]

        comisiones_2024.loc[i, ['comision_%']] = bonos_limites_comisiones_line[['%']].iloc[0].values
        comisiones_2024.loc[i, ['comision_$']] = comisiones_2024.loc[i, ['comision_%']].iloc[0] * comisiones_2024.loc[i, ['utilidad']].iloc[0]
        comisiones_2024.loc[i, ['bono_venta_$']] = bonos_limites_comisiones_line[['bono']].iloc[0].values


    return comisiones_2024

In [ ]:
comisiones_1_2024 = comisiones_mes_2024_func(1, costo_ventas_list)
comisiones_2_2024 = comisiones_mes_2024_func(2, costo_ventas_list)
comisiones_3_2024 = comisiones_mes_2024_func(3, costo_ventas_list)
comisiones_4_2024 = comisiones_mes_2024_func(4, costo_ventas_list)
comisiones_5_2024 = comisiones_mes_2024_func(5, costo_ventas_list)
comisiones_6_2024 = comisiones_mes_2024_func(6, costo_ventas_list)
comisiones_7_2024 = comisiones_mes_2024_func(7, costo_ventas_list)

comisiones_2024 = pd.concat([
                        comisiones_1_2024,
                        comisiones_2_2024,
                        comisiones_3_2024,
                        comisiones_4_2024,
                        comisiones_5_2024,
                        comisiones_6_2024,
                        comisiones_7_2024
                    ])

In [ ]:
with engine.connect() as conn, conn.begin():  
    comisiones_2024.to_sql(name = 'comisiones_2024', index= False, con = conn, if_exists = 'replace')

engine.dispose()

print('Todas las tablas se ingresaron con éxito')

In [ ]:
cobranza = cobranza_enero.copy()
costo_ventas = costo_ventas_enero.copy()

cobranza.loc[:, ['año']] = cobranza['fact_doc_date'].dt.year
cobranza.loc[:, ['mes']] = cobranza['fact_doc_date'].dt.month

cobranza_paga = cobranza.loc[
                            cobranza['paga_comision'] == True,
                            [
                                'salesperson_id', 'pay_date', 'partner_id','fact_doc_date',  'año', 'mes'
                            ]
                        ]

costo_ventas.loc[:, ['mes']] = costo_ventas['invoice_date'].dt.month

cobranza_paga

In [ ]:
wep = cobranza_año.loc[
            (cobranza_año['paga_comision'] == True)
            & (cobranza_año['salesperson_id'] == 211)
            , 
            ['fact_doc_name', 'fact_doc_date', 'pay_date', 'dias_pago']
        ]

wep['año'] = wep['fact_doc_date'].dt.year
wep['mes'] = wep['fact_doc_date'].dt.month

wep.groupby(['año', 'mes']).count()